In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Question Answering with Generative Models on Vertex AI


<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/question_answering.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/question_answering.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/question_answering.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

Large language models can be used for various natural language processing tasks, including question-answering (Q&A). These models are trained on a vast amount text data and can generate high-quality responses to a wide range of questions. One thing to note here is that most models have cutoff dates regarding their knowledge, and asking anything too recent might yield an incomplete, imaginative or incorrect answer (i.e. a hallucination).

This notebook covers the essentials of prompts for answering questions using a generative model. In addition, it showcases the `open domain` (knowledge available on the public internet) and `closed domain` (knowledge that is more private - typically enterprise or personal knowledge).

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview#prompt_structure).

### Objective

By the end of the notebook, you should be able to write prompts for the following:

* **Open domain** questions:
    * Zero-shot prompting
    * Few-shot prompting


* **Closed domain** questions:
    * Providing custom knowledge as context
    * Instruction-tune the outputs
    * Few-shot prompting

## Getting Started

### Install Vertex AI SDK

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user

### Install additional Python Packages

In [ ]:
!pip install -q python-Levenshtein --upgrade --user
!pip install -q fuzzywuzzy --upgrade --user

The following cell restarts the notebook kernel. For Vertex AI Workbench you can restart from the terminal using the kernel status button on top. 

In [ ]:
# Automatically restart kernel after package installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Import libraries

**Colab only:** Uncomment the following cell to initialize the Vertex AI SDK. For Vertex AI Workbench, you don't need to run this.  

In [ ]:
# import vertexai

# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import pandas as pd
from vertexai.preview.language_models import TextGenerationModel

### Import models

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## Question Answering

Question-answering capabilities require providing a prompt or a question that the model can use to generate a response. The prompt can be a few words or a few complete sentences, depending on the complexity of the question.

When creating a question-answering prompt, it is essential to be specific and provide as much context as possible. It helps the model understand the intent behind the question and generate a relevant response. For example, if you want to ask:

```
"What is the capital of France?",

then a good prompt could be:

"Please tell me the name of the city that serves as the capital of France."

```

In addition to being specific, the prompt should also be grammatically correct and free of spelling errors. It helps the model generate a response that is easy to understand and contains fewer errors or inaccuracies.

By providing specific, context-rich prompts, you can help the model understand the intent behind the question and generate accurate and relevant responses.


Below are some differences between the **open domain** and **closed domain** categories for question-answering prompts.

* **Open domain**: All questions whose answers are available online already. They can belong to any category, like history, geography, countries, politics, chemistry, etc. These include trivia or general knowledge questions, like:

```
Q: Who won the Olympic gold in swimming?
Q: Who is the President of [given country]?
Q: Who wrote [specific book]"?
```

Keep in mind the training cutoff of generative models, as questions involving information more recent than what the model was trained on might give incorrect or imaginative answers.


* **Closed domain**: If you have some internal knowledge base not available on the public internet, then those belong to the _closed domain_ category.
You can pass that "private" knowledge as context to the model. If prompted correctly, the model is more likely to answer from within the context provided and less likely to give answers beyond that from the open internet.

Consider the example of building a Q&A bot over your internal product documentation. In this case, you can pass the complete documentation to the model and prompt it only to answer based on that.

Typical prompt for **closed domain**:

```
Prompt: f""" Answer from the below context: \n\n
		   context: {your knowledge base} \n
		   question: {question specific to that knowledge base}  \n
		   answer: {to be predicted by model} \n
		"""
```

Below are some examples to understand these different types of prompts.

### Open Domain

#### Zero-shot prompting

In [ ]:
prompt = """Create a catchy hashtag based on this house description: \"Coastal Elegance Awaits! Immerse yourself in beach living with this stunning 4-bedroom, 3-bathroom home. Recently renovated with over $200,000 in upgrades, this move-in ready haven offers the perfect blend of style and functionality.
Step inside to discover a beautifully updated kitchen featuring quartz counters, stainless steel appliances, and a vent hood. The spacious dining area leads to a cozy formal living room with a wood-burning fireplace, creating a welcoming space for gatherings.
All three bathrooms have been completely remodeled, showcasing new vanities, quartz counters, and elegant tubs/showers with glass enclosures.
The master bedroom addition provides an expansive retreat with a walk-in closet, while a second master bedroom at the front offers versatility.
This delightful home boasts bonus features like all-new LVT flooring, smooth ceilings, recessed lighting, dual-pane windows, and a Nest thermostat for modern comfort.
Outside, the private rear yard is bordered by block wall fencing, providing a serene space to relax or entertain. A large side yard offers additional opportunities for outdoor enjoyment.
Situated in a quiet neighborhood, the property is within walking distance to award-winning schools, a park, golf, shopping, and restaurants. Plus, you\'re only a 5-minute drive from the beach and the vibrant Lovely Mall.
Don\'t miss out on this coastal gem with luxurious upgrades and a convenient location. Schedule a showing today and experience the beach lifestyle you\'ve been dreaming of!\"
"""

print(
    generation_model.predict(
        prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

#### Few-shot prompting

Let's say you want to a get a short answer from the model (like only a specific name). To do so, you can leverage a few-shot prompt and provide examples to the model to illustrate the expected behavior.

In [ ]:
prompt = """Your real estate company aims to establish a strong social media presence and is eager to create a catchy hashtag based on long house descriptions in ads.


input: Introducing 1234 Seaside Drive, a beautifully remodeled 2-story home with breathtaking ocean views from both levels. The upper level boasts an inviting open floor plan with vaulted beamed ceilings and a charming fireplace. The kitchen features a stunning oversized quartz island and top-of-the-line appliances. The luxurious primary suite offers ocean vistas, vaulted ceilings, and an en-suite bath with dual sinks and a glass-tiled walk-in shower. Downstairs, two generously sized bedrooms with ocean views and a stylish bathroom await. Step outside to the expansive patio, perfect for entertaining amidst a tasteful succulent garden. Situated in the sought-after Coastal Heights neighborhood, just minutes from shops, dining, beaches, and natural parks. Your dream coastal retreat awaits!


output: #CoastalVistaRetreat


input: Experience beach resort living in this carefree condo nestled on a hill with panoramic views of the Horizon Bay Resort. Immerse yourself in the serene sounds of waves at Sandy Cove and Sunset Beach, just a stone\'s throw away across Ocean Breeze Highway.
This two-bedroom upper level condo offers an open floor plan, custom lighting, and a breathtaking ocean view from your private deck. Indulge in the remodeled kitchen with sleek stainless steel appliances and stylish granite countertops. The bathroom boasts impeccable quality, adding a touch of luxury to your coastal retreat.
Both bedrooms feature ceiling fans and ample natural light, offering tranquil views of the coastal hills. Storage is a breeze with the shaded, over-sized carport area and large attic space with a pull-ladder.
Relax at the resort-quality association pool, complete with lounge chairs, umbrellas, and two restrooms with showers, all while enjoying sweeping views over Bluewater Creek and the glistening Pacific.
Enjoy an active lifestyle with nearby yoga in the park, golf, hiking trails, shops, and dining. Multiple luxury resorts are mere minutes away.
Hop aboard the Seaside Express and Coastal Breeze trolleys, conveniently stopping at the base of the hill, offering easy access to summer concerts, festivals, and vibrant coastal attractions.
Don\'t miss this chance to make your seaside dreams come true! Contact us now for a private showing.
output: #SeasideEscape


input: Coastal Elegance Awaits! Immerse yourself in beach living with this stunning 4-bedroom, 3-bathroom home. Recently renovated with over $200,000 in upgrades, this move-in ready haven offers the perfect blend of style and functionality.
Step inside to discover a beautifully updated kitchen featuring quartz counters, stainless steel appliances, and a vent hood. The spacious dining area leads to a cozy formal living room with a wood-burning fireplace, creating a welcoming space for gatherings.
All three bathrooms have been completely remodeled, showcasing new vanities, quartz counters, and elegant tubs/showers with glass enclosures.
The master bedroom addition provides an expansive retreat with a walk-in closet, while a second master bedroom at the front offers versatility.
This delightful home boasts bonus features like all-new LVT flooring, smooth ceilings, recessed lighting, dual-pane windows, and a Nest thermostat for modern comfort.
Outside, the private rear yard is bordered by block wall fencing, providing a serene space to relax or entertain. A large side yard offers additional opportunities for outdoor enjoyment.
Situated in a quiet neighborhood, the property is within walking distance to award-winning schools, a park, golf, shopping, and restaurants. Plus, you\'re only a 5-minute drive from the beach and the vibrant Lovely Mall.
Don\'t miss out on this coastal gem with luxurious upgrades and a convenient location. Schedule a showing today and experience the beach lifestyle you\'ve been dreaming of!
output:
"""

print(
    generation_model.predict(
        prompt,
        max_output_tokens=20,
        temperature=0.1,
    ).text
)

#### Zero-shot prompting vs Few-shot prompting

Zero-shot prompting can be useful for quickly generating text for new tasks, but the quality of the generated text may be lower than that of a few-shot prompt with well-chosen examples. Few-shot prompting is typically better suited for tasks that require a high degree of specificity or domain-specific knowledge, but requires some additional thought and potentially data to set up the prompt.

### Closed Domain

#### Adding internal knowledge as context in prompts

Imagine a scenario where you would like to build a question-answering bot that takes in internal documentation and lets users ask questions about it.

In the example below, the Google Cloud Storage and content policy documentation is added to the prompt, so that the PaLM API can use that to answer subsequent questions with the provided context.

In [ ]:
context = """
Creating catchy hashtags for a real estate company involves understanding your target audience, being creative, and aligning with your brand identity. Here are some tips to help you create compelling and effective hashtags:
Keep them short and sweet. Hashtags should be easy to remember and type, so keep them to 1-2 words or a short phrase.
Use relevant keywords. When people are searching for real estate, they'll likely use relevant keywords in their search terms. Make sure to include these keywords in your hashtags so that your listings will show up in their search results.
Be creative. Don't be afraid to get creative with your hashtags. Use puns, wordplay, or other creative techniques to make your hashtags stand out.
Use trending hashtags. If there are any trending hashtags related to real estate, use them in your posts. This will help your listings get seen by more people.
Use branded hashtags. Create your own branded hashtags that you can use across all of your social media channels. This will help people to identify your brand and remember your listings.
"""

question = "What is the best way to create a catchy hashtag based on a real estate description ?"

prompt = f"""Answer the question given in the context below:
Context: {context}?
Question: {question} 
Answer:
"""

print("[Prompt]")
print(prompt)

print("[Response]")
print(
    generation_model.predict(
        prompt,
    ).text
)


#### Instruction-tuning outputs

Another way to help out language models is to provide additional instructions to frame the output in the prompt. To ensure the model doesn't respond to anything outside the context, the prompt can specify that the response should be "Information not available in provided context" if that's the case.

In [ ]:
question = "Tell me the current market price of a specific property in a particular city?"
prompt = f"""Answer the question given the context below as {{Context:}}. \n
If the answer is not available in the {{Context:}} and you are not confident about the output,
please say "Information not available in provided context". \n\n
Context: {context}?\n
Question: {question} \n
Answer:
"""

print("[Prompt]")
print(prompt)

print("[Response]")
print(
    generation_model.predict(
        prompt,
        max_output_tokens=256,
        temperature=0.3,
    ).text
)


#### Few-shot prompting

In [ ]:
prompt = """
Context:
As the housing market experiences fluctuations in demand, supply, and economic conditions, the concept of "rent-to-own" arrangements has gained popularity among both tenants and homeowners. Rent-to-own, also known as lease-to-own or rent-to-buy, is a housing arrangement that offers potential buyers an alternative path to homeownership, particularly for those who may face challenges in qualifying for traditional mortgages or are not ready for an immediate purchase.

Question:
What is a rent-to-own housing arrangement, and how does it work for both tenants and homeowners?

Answer:
A rent-to-own housing arrangement is a contract between a tenant and a homeowner where the tenant has the option to buy the property at a predetermined price after a specified rental period. It offers potential buyers a pathway to homeownership while providing homeowners with a secure rental income and a potential future sale.
---

Context:
The concept of "housing bubbles" was first widely discussed in the early 2000s after a series of real estate market crashes in various countries. Since then, the phenomenon of housing bubbles has become a significant topic of interest in the housing market, with potential implications for both buyers and sellers.

Question:
When were housing bubbles first widely discussed?

Answer:
Housing bubbles were first widely discussed in the early 2000s after a series of real estate market crashes in various countries.

---

Context:
With the rapid growth of social media platforms and their influence on the real estate industry, the use of housing hashtags has become a crucial marketing strategy for real estate agents and property sellers. Crafting the right hashtags can significantly impact the visibility and reach of property listings, helping to attract potential buyers and generate interest in the housing market.

Question: Why are housing hashtags so crucial ?

Answer:

"""
print(
    generation_model.predict(
        prompt,
    ).text
)


### Evaluation

You can evaluate the outputs of the question and answering task if the ground truth answers of each question are available. In zero-shot prompting, you can only use `open domain` questions. However, with `closed domain` questions, you can add context and evaluate similarly.  To showcase how that will work, start by creating a simple dataframe with questions and ground truth answers. 

In [ ]:
qa_data = {
    "question": [
        "What is a mortgage?",
        "What is a condo?",
        "What is a duplex?",
    ],
    "answer_groundtruth": ["a loan secured by a mortgage", "a type of housing", "a building with two separate units"],
}
qa_data_df = pd.DataFrame(qa_data)
qa_data_df


Now that you have the data with questions and ground truth answers, you can call the PaLM 2 generation model to each review row using the `apply` function. Each row will use the dynamic prompt to predict the answer using the PaLM API. We will save the results in `answer_prediction` column.  


In [ ]:
def get_answer(row):
    prompt = f"""Answer the following question as precise as possible.\n\n
            question: {row}
            answer:
              """
    return generation_model.predict(
        prompt=prompt,
    ).text


qa_data_df["answer_prediction"] = qa_data_df["question"].apply(get_answer)
qa_data_df

You may want to evaluate the answers predicted by the PaLM API. However, it will be more complex than the text classification since the answers may differ from ground truth and may be presented in slightly more/fewer words. 

For example, you can observe the question "What is the name of the Earth's largest ocean?" and see that model predicted  "Pacific Ocean" when a ground truth label is "The Pacific Ocean" with the extra "The." Now, if you use the simple classification metrics, then you will consider this as a wrong prediction since original and predicted strings have a difference. However, you can see that the answer is correct since an extra "The" is causing the issue. It's a simple string comparison problem.

The solution to string comparison where both `ground_thruth` and `predicted` may have some extra or fewer letters, one approach is to use a fuzzy matching algorithm. 
Fuzzy string matching uses [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance) to calculate the differences between two strings. 

For example, the Levenshtein distance between "kitten" and "sitting" is 3, since the following 3 edits change one into the other, and there is no way to do it with fewer than 3 edits:

* kitten → sitten (substitution of "s" for "k"),
* sitten → sittin (substitution of "i" for "e"),
* sittin → sitting (insertion of "g" at the end).


Here's another example, but this time using `fuzzywuzzy`  library, which gives us the same `Levenshtein distance` between two strings but in ratio. The ratio raw score measures the string's similarity as an int in the range [0, 100]. For two strings X and Y, the score is defined by int(round((2.0 * M / T) * 100)) where T is the total number of characters in both strings, and M is the number of matches in the two strings. 

Read more here about the [ratio formula](https://anhaidgroup.github.io/py_stringmatching/v0.3.x/Ratio.html) : 

You can see one example to understand this furhter. 
```
String1: "this is a test"
String2: "this is a test!"

Fuzz Ratio => 97  #

Fuzz Partial Ratio => 100  #Since most characters are the same and in a similar sequence, the algorithm calculates the partial ratio as 100 and ignores simple additions (new characters). 
```


Now compute a score to perform fuzzy matching:

In [ ]:
from fuzzywuzzy import fuzz


def get_fuzzy_match(df):
    return fuzz.partial_ratio(df["answer_groundtruth"], df["answer_prediction"])


qa_data_df["match_score"] = qa_data_df.apply(get_fuzzy_match, axis=1)
qa_data_df

Now that you have the individual match score (partial), you can take the mean or average of the whole column to get a sense of overall data. 
Scores closer to 100 mean PaLM 2 can predict closer to ground truth; if the score is towards 50 or 0, it did not perform well.

In [ ]:
print(
    "the average match score of all predicted answer from PaLM 2 is : ",
    qa_data_df["match_score"].mean(),
    " %",
)

In this case, you get 100% as the mean score, even though some predictions were missing some words. That means you are very close to the ground truth, and some answers are just missing the exact verboseness of the ground truth. 